In [1]:
import os
from datetime import datetime
import time
import pyaudio
import librosa
import asyncio
import threading
#import deepspeech
import wave
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, AutoProcessor, AutoModelForCTC, TFWav2Vec2ForCTC, \
                         AutoTokenizer, AutoFeatureExtractor, T5ForConditionalGeneration, T5Tokenizer
import spacy
from spellchecker import SpellChecker
#from autocorrect import Speller
import openai

import torch
import torchaudio
import tensorflow

In [4]:
#TODO когда санкции снимут
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./google_cred.json"

## ПРЕДИСЛОВИЕ

#### Перед тем, как запускать прогу, необходимо:

1) установить virtual audio cable

2) в sound mixer options для всех бырать defaul input = Linme 1(virtual audio cable)

3) затем для нужной программы (окна) в oputput выбрать тот же кабель (Line 1, например) 

4) вызвать код для нахождения подключённых аудиоустройств: ```for i in range(a.recorder.get_device_count()):
       print(i, a.recorder.get_device_info_by_index(i))```

5) выбрать подходящее аудиоустройство (для записи должно быть устройство Line 1, у которого ```'maxInputChannels' != 0```,
  а для вывода должно быть устройство Line 1, у которого ```'maxOutputChannels' != 0```

6) и когда открываете потоки в kwargs передаёте нужный вам индекс в переменную ```input_device_index```

7) после запуска у нас звук будет перенаправляться так: приложение->Line_1_output->Line_1_input->Наша прога.

In [2]:
class AudioChanel():
    def __init__(self):
        #change this according to your model
        self.chunk = 1024
        self.format = pyaudio.paInt16
        self.channels = 1#2
        self.rate = 16000
        self.recorder = pyaudio.PyAudio()
        self.frames = []
        self.seconds = 2
        # Открыть поток для записи
        #1 {'index': 1, 'structVersion': 2, 'name': 'Line 1 (Virtual Audio Cable)', 'hostApi': 0, 'maxInputChannels': 2, 
        #   'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 
        #   'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
        self.input_stream = self.recorder.open(format=self.format,
                                          channels=self.channels,
                                          rate=self.rate,
                                          input=True,
                                          frames_per_buffer=self.chunk,
                                          input_device_index=1) #0,1 - work
        # Открыть поток для воспроизведения
        #5 {'index': 5, 'structVersion': 2, 'name': 'Line 1 (Virtual Audio Cable)', 'hostApi': 0, 'maxInputChannels': 0, 
        #   'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 
        #   'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
        self.output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        
        
    def record(self):
        while True:
            data = self.input_stream.read(self.chunk)
            self.frames.append(data)

    def play(self, audio = None):
        output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        match audio:
            case list():
                for frame in audio:
                    output_stream.write(frame)
            case _:
                output_stream.write(audio.tobytes())

    def play_default(self):
        output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        with threading.Lock():
            to_play = list(self.frames)
        for frame in to_play:
            output_stream.write(frame)
    
    def get(self):
        with threading.Lock():
            return list(self.frames)
        
        
    def close(self):
        self.input_stream.stop_stream()
        self.input_stream.close()

In [ ]:
class Agent():
    def __init__(self):
        self.audio = AudioChanel()
        self.device = torch.device('cuda')
        #you may use your models instead
        self.processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
        self.model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(self.device)
        self.feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

        self.spell = SpellChecker()
        self.nlp = spacy.load('en_core_web_sm')#en_core_web_trf
        self.PASS_VAR = False

        #the worst model EVER
        model_name = 'utrobinmv/t5_translate_en_ru_zh_small_1024'# small, large and base
        self.model_translate = T5ForConditionalGeneration.from_pretrained(model_name).to(self.device)
        self.tokenizer_translate = T5Tokenizer.from_pretrained(model_name)
        
        language = 'ru'
        model_id = 'v4_ru'
        self.sample_rate = 48000
        self.speaker = 'kseniya'
        
        self.model_voice, _ = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                             model='silero_tts',
                                             language=language,
                                             speaker=model_id)
        self.model_voice.to(self.device)
        self.to_play = []
    
    def correcting(self,sent):
        to_spell = sent.split()
        corrected_words = [self.spell.correction(word) for word in to_spell]
        return ' '.join((word if word is not None else to_spell[idx]) for idx, word in enumerate(corrected_words))

    def send_to_play(self):
        while True:
            time.sleep(1)
            with threading.Lock():
                copy_to_play = list(self.to_play)
            while len(copy_to_play):
                to_play = copy_to_play[0]
                translation = threading.Thread(target=self.audio.play,args=(to_play[0],))
                original = threading.Thread(target=self.audio.play,args=(to_play[1],))
                
                translation.start()
                original.start()
                
                translation.join()
                original.join()
                with threading.Lock():
                    del self.to_play[0]
                    del copy_to_play[0]
    
    async def run(self):
        #start recording
        _ = threading.Thread(target=self.audio.record).start()
        _ = threading.Thread(target=self.send_to_play).start()
        
        # await asyncio.sleep(10)
        last_state = ''
        while True:
            '''
            PART 1
            RECORDING
            description: record every 2 seconds of audio from PC
            '''
            await asyncio.sleep(2)
            time = datetime.now()
            frames = self.audio.get()[:200]
            to_model = np.frombuffer(b''.join(frames), dtype=np.int16)
            end_1 = datetime.now()
            # print('end_1: ', end_1)
            '''
            PART 2
            CONVERTING AUDIO TO TEXT
            description: put audio into model and get recognized words
            '''
            input_values = self.processor(to_model, sampling_rate = self.audio.rate, return_tensors="pt").input_values  # Batch size 1
            logits = self.model(input_values.to(torch.float).to(self.device)).logits
            predicted_ids = torch.argmax(logits, axis=-1)
            
            outputs = self.tokenizer.decode(predicted_ids[0], output_word_offsets=True)
            # compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
            time_offset = self.model.config.inputs_to_logits_ratio / self.feature_extractor.sampling_rate
            
            word_offsets = [
                {
                    "word": d["word"],
                    "start_time": round(d["start_offset"] * time_offset, 2),
                    "end_time": round(d["end_offset"] * time_offset, 2),
                }
                for d in outputs.word_offsets
            ]
            end_2 = datetime.now()
            print('end_2: ', end_2-end_1)
            '''
            PART 3
            FIND SENTENCIES IN THE OUTPUT TEXT
            description: find grammar parts in output text for building whole sentences
            '''
            text = ' '.join([word['word'] for word in word_offsets])
            fixed = self.correcting(text)
            print('Full recorded text: ', text)
            if last_state == '':
                last_state = text
            else:
                if last_state == text:
                    self.PASS_VAR = True
                last_state = text
                
            doc = self.nlp(fixed)
    
            end_3 = datetime.now()
            print('end_3: ', end_3-end_2)
            '''
            PART 4
            TRANSLATE SENTENCE INTO RUSSIAN (or any other language)
            description: translate one of the list, because we should be sure about full context
            '''
            sents = list(doc.sents)
            if len(sents)<=1 and not self.PASS_VAR:
                continue
            self.PASS_VAR = False
            print(sents)
            
            prefix = 'translate to ru: '
            ### delete part of audio.frames list
            with threading.Lock():
                idx_del = len(sents[0].text.split())-1 if len(sents[0].text.split())==len(word_offsets) else len(sents[0].text.split())
                to_del = int(word_offsets[idx_del]['end_time']*self.audio.rate/self.audio.chunk)
                play_background = self.audio.frames[:to_del+1]
                del self.audio.frames[:to_del]
            ###
            src_text = prefix + sents[0].text
            
            # translate Englosh to Russian
            input_ids = self.tokenizer_translate(src_text, return_tensors="pt")
            
            generated_tokens = self.model_translate.generate(**input_ids.to(self.device))
            
            result = self.tokenizer_translate.batch_decode(generated_tokens, skip_special_tokens=True)
            end_4 = datetime.now()
            print('end_4: ', end_4-end_3)
            '''
            PART 5
            VOICEOVER OF TEXT AND PLAY
            description: voice translated text
            '''
            print(result[0])
            if not len(result[0]):
                continue
                  
            audio = self.model_voice.apply_tts(text=result[0],
                        speaker=self.speaker,
                        sample_rate=self.sample_rate)
            
            audio_numpy = audio.cpu().numpy()
            audio_resampled = librosa.resample(audio_numpy, orig_sr=self.sample_rate, 
                                               target_sr=self.audio.rate, res_typestr='fft')
            audio_int16 = (audio_resampled * 32767).astype(np.int16)
            end_5 = datetime.now()
            print('end_5: ', end_5-end_4)
            # self.audio.play(audio_int16)
            with threading.Lock():
                self.to_play.append((audio_int16,frames))
            # _ = threading.Thread(target=self.audio.play, args=(play_background,)).start()
            
a = Agent()
await a.run()

E:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initiali

end_2:  0:00:01.958723
Full recorded text:  ESSE JAE NOT KNOWING WHET
end_3:  0:00:00.008266
end_2:  0:00:00.201824
Full recorded text:  YES SE JANE E NOT KNOWING WHET WE GURS MAKING LAL TAME SIE WILL SAY JUST I ID E
end_3:  0:00:00.010520
end_2:  0:00:00.189306
Full recorded text:  ES SE JANE NOT KNOWING WHET WIGORS MAKING MALL TANEVYSIE WILL SAY JUST IIT IDIT YOUDON'T WAYS KNOW OUR PEATUR WY I SO
end_3:  0:00:01.754830
[ES we JANE NOT KNOWING WHET rigors MAKING MALL TANEVYSIE WILL SAY JUST it idiot YOUDON'T WAYS KNOW, OUR peanut, we, I, SO]
end_4:  0:00:01.027821
Мы, Дженни, не знаем, какие строгие требования делают БОЛЬШИЕ ТАНЕВИСИИ, СКАЗАТЬ: "Ты просто идиот, что ты не знаешь".
end_5:  0:00:07.182376
end_2:  0:00:00.271984
Full recorded text:  HE SHAR BY HI TAR AN HE WAS REE THOSTIN HER AND LIKS LOR IN HIS OLD BESIDIN WE HAVE THE PLAN THAT THE SALO WAS GOING TO BE AWAY FOR PEOPL
end_3:  0:00:00.986574
[HE share BY HI TAR, AN HE WAS see hosting HER AND like for IN HIS OLD besides WE